In [ ]:
from google.colab import drive
drive.mount('/content/drive')
URL = "/content/drive/MyDrive/235757_HAICon2021_dataset.zip (Unzipped Files)"

Mounted at /content/drive


In [ ]:
!pip install ./eTaPR-21.8.2-py3-none-any.whl

Processing ./eTaPR-21.8.2-py3-none-any.whl


In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange
#from TaPR_pkg import etapr
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import time
from scipy import signal

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:

TRAIN_DATASET = sorted([x for x in Path(URL+"/train/").glob("*.csv")])
TRAIN_DATASET

VALIDATION_DATASET = sorted([x for x in Path(URL+"/validation/").glob("*.csv")])
VALIDATION_DATASET

TEST_DATASET = sorted([x for x in Path(URL+"/test/").glob("*.csv")])
TEST_DATASET

[PosixPath('/content/drive/MyDrive/235757_HAICon2021_dataset.zip (Unzipped Files)/test/test1.csv'),
 PosixPath('/content/drive/MyDrive/235757_HAICon2021_dataset.zip (Unzipped Files)/test/test2.csv'),
 PosixPath('/content/drive/MyDrive/235757_HAICon2021_dataset.zip (Unzipped Files)/test/test3.csv')]

In [ ]:
def dataframe_from_csv(target):
    return pd.read_csv(target, engine='python').rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

In [ ]:
TRAIN_DF_RAW = dataframe_from_csvs(TRAIN_DATASET)
TRAIN_DF_RAW


,timestamp,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,...,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86
0,2021-07-11 10:00:00,-2.2642,0,12.26196,-0.00087,12.01019,0.2254,12.66931,70,1,50,11704,17.61017,28.03686,27170,27.24188,25.42843,0,1,50,27.28778,31.02721,1,267.90363,467.27020,3865,50,14742,0.88504,1,54816,1009.15302,58.21227,5.5352,0,126.92957,2880,26.92264,0,0,...,48.64655,1,0,13752,41.81518,1,0.14465,0.46783,0,276.45758,476.76703,8.90254,49.26452,719.96625,1,923,0,0,16,0.08771,15.07713,9914,1,306.07104,28.20367,15.50464,251.23593,451.06253,26.13983,48.90190,0.92255,30.08042,7.08818,595.06104,276.40338,1,1014.79321,12.0,50,3506
1,2021-07-11 10:00:01,-2.4923,0,12.26196,0.00058,12.56714,-0.0711,12.66931,70,1,50,11847,17.61780,28.03169,27171,27.24188,25.42656,0,1,50,27.28778,31.01196,1,268.95258,466.91238,3898,50,14781,0.88619,1,54816,1011.06647,57.44932,4.7507,0,126.92957,2880,26.92264,0,0,...,50.40893,1,0,13752,41.81518,1,0.14465,0.42892,0,278.91705,476.76703,8.90254,49.26452,720.10663,1,923,0,0,16,0.08771,14.97197,9914,1,303.09189,28.19449,16.20023,250.47298,404.38739,26.13983,48.94870,0.92255,30.08423,7.08818,531.50317,276.18634,1,1014.79321,12.0,50,3493
2,2021-07-11 10:00:02,-2.8460,0,12.26196,-0.00072,14.48975,0.0051,12.66931,70,1,50,11953,17.61780,28.03504,27170,27.24188,25.41979,0,1,50,27.28778,31.01196,1,269.76636,466.91238,3892,50,14831,0.88836,1,54816,1009.79083,58.78445,5.5709,0,126.92957,2880,26.92264,0,0,...,50.60730,1,0,13760,41.81518,1,0.14465,0.42282,0,278.89899,476.76703,8.90254,49.26452,719.96625,1,923,0,0,16,0.08771,14.90129,9914,1,299.11945,28.18530,16.53352,249.90080,382.53925,26.14746,48.89090,0.91873,30.09148,7.08818,451.06253,279.85754,1,1014.79321,12.0,50,3490
3,2021-07-11 10:00:03,-2.1235,0,12.26196,0.00101,15.93170,0.1842,12.66931,70,1,50,12051,17.61780,28.03510,27171,27.24188,25.44255,0,1,50,27.28778,31.01196,1,271.64716,466.35010,3876,50,14917,0.89214,1,54816,1008.77380,59.16594,5.1828,0,126.92957,2880,26.92264,0,0,...,49.14246,1,0,13760,41.81518,1,0.14465,0.43121,0,280.49048,476.76703,8.90254,49.26452,720.00159,1,923,0,0,16,0.08771,14.82246,9914,1,306.07104,28.17612,16.30247,250.47298,396.44250,26.13983,48.95938,0.91644,30.10407,7.08818,404.38739,281.50317,1,1014.79321,12.0,50,3525
4,2021-07-11 10:00:04,-2.9074,0,12.26196,0.00043,16.10718,0.1842,12.66931,70,1,50,12180,17.61780,28.03510,27171,27.24188,25.44007,0,1,50,27.28778,31.02721,1,272.80457,466.60565,3871,50,14997,0.89191,1,54815,1009.79083,58.78445,5.1828,0,126.92957,2880,26.92264,0,0,...,47.18170,1,0,13760,41.81518,1,0.14465,0.45486,0,283.47437,476.76703,8.90254,49.26452,720.00159,1,923,0,0,16,0.08771,14.73087,9914,1,308.05728,28.16693,15.79237,250.28227,427.22849,26.13983,49.02211,0.91797,30.10331,7.08818,382.53925,281.34039,1,1014.79321,12.0,50,3503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259195,2021-08-09 08:59:56,-2.0065,0,12.26196,0.00051,100.28228,0.5977,12.53358,70,1,50,8536,13.45978,28.03433,27213,27.75778,37.62007,0,1,50,27.70997,31.40868,1,318.30511,465.27664,2,50,17293,1.08731,1,54740,991.81366,58.78445,5.7302,0,112.42368,2880,31.38997,0,0,...,12.39929,1,0,0,36.39832,1,64.39886,0.36179,0,307.83423,459.18994,4.67615,49.62311,720.04309,1,905,0,0,13856,0.13800,100.00000,10000,1,308.05728,35.00000,100.00000,242.46214,3175.36084,38.49182,48.07038,1.17355,35.05434,10.00000,3178.48877,370.02679,1,986.05908,12.0,50,136
259196,2021-08-09 08:59:57,-2.2101,0,12.26196,-0.00029,100.28228,0.5509,12.53358,70,1,50,8519,13.45978,28.03521

In [ ]:
del TRAIN_DF_RAW['timestamp']
T_corr = TRAIN_DF_RAW.corr()
T_corr

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,...,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86
C01,1.000000,NaN,-0.445858,-0.001231,-0.408117,0.963465,-0.391064,-0.500685,NaN,NaN,-0.123100,-0.000409,0.646109,-0.241069,0.224054,-0.104969,-0.011650,NaN,NaN,0.289443,-0.229453,NaN,0.019033,-0.000216,-0.011659,NaN,0.023108,-0.031692,NaN,0.032666,-0.053855,-0.064687,0.869700,-0.011650,0.111973,NaN,-0.425874,NaN,NaN,-0.405410,...,0.357106,0.011650,NaN,-0.003682,0.315344,NaN,-0.012439,0.111045,NaN,0.017259,0.000327,0.042461,-0.065719,0.064525,0.011650,0.108083,NaN,-0.011650,-0.012366,-0.128160,-0.402887,-0.052933,NaN,-0.065036,-0.027854,-0.408683,-0.058691,-0.403582,-0.102619,-0.065862,-0.039759,-0.421752,0.176072,-0.403591,0.004099,NaN,-0.054186,-0.002531,NaN,0.004918
C02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C03,-0.445858,NaN,1.000000,0.000854,0.403762,-0.484655,0.307819,0.369021,NaN,NaN,0.139525,0.014064,-0.328341,0.199756,-0.146445,0.094967,0.203931,NaN,NaN,-0.128146,0.194048,NaN,-0.016948,0.104889,0.005527,NaN,-0.019834,0.009190,NaN,-0.197047,0.029588,0.035604,-0.367775,0.203925,-0.118139,NaN,0.414239,NaN,NaN,0.388818,...,-0.342419,-0.203925,NaN,-0.000287,-0.311470,NaN,0.008383,-0.112579,NaN,-0.015237,0.106623,-0.280845,-0.031060,-0.027506,-0.203931,-0.238652,NaN,0.203931,0.008254,0.066512,0.386090,0.021569,NaN,0.035726,0.107409,0.404092,0.028690,0.386639,0.093740,-0.030192,0.012343,0.410523,-0.233239,0.386640,-0.004852,NaN,0.029897,0.113620,NaN,-0.007610
C04,-0.001231,NaN,0.000854,1.000000,0.000789,-0.001517,0.001340,0.000976,NaN,NaN,0.003819,0.008544,-0.002440,0.001382,0.000110,0.023758,-0.000206,NaN,NaN,-0.000014,0.000918,NaN,0.115876,-0.000789,-0.036402,NaN,0.104251,0.115101,NaN,0.000494,0.000484,0.014615,-0.001672,-0.000200,-0.000015,NaN,0.001383,NaN,NaN,0.000667,...,-0.000642,0.000200,NaN,-0.041368,-0.001439,NaN,0.031887,0.001586,NaN,0.095780,0.000164,-0.001011,0.000740,0.035503,0.000206,0.000379,NaN,-0.000206,0.038185,-0.000050,0.000984,0.004658,NaN,0.014099,-0.000217,0.000858,0.000537,0.000720,0.014141,0.000609,0.003957,0.015236,-0.001083,0.000695,0.086206,NaN,0.000282,0.000264,NaN,-0.062514
C05,-0.408117,NaN,0.403762,0.000789,1.000000,-0.394264,0.267740,0.406159,NaN,NaN,0.091949,0.027560,-0.393498,0.628777,0.051627,0.313173,0.042431,NaN,NaN,-0.077117,0.625267,NaN,-0.007650,-0.003049,0.002201,NaN,-0.016391,0.045292,NaN,-0.061443,0.093311,0.105578,-0.366846,0.042434,-0.085890,NaN,0.944182,NaN,NaN,0.980008,...,-0.832047,-0.042434,NaN,-0.008314,-0.713814,NaN,0.024368,-0.338478,NaN,-0.007667,-0.002348,-0.083787,0.099896,0.092221,-0.042431,-0.125429,NaN,0.042431,0.024151,0.145896,0.966856,0.066027,NaN,0.106195,0.326016,0.999727,0.099786,0.968004,0.313098,0.099212,0.056144,0.938925,-0.018297,0.967722,0.007703,NaN,0.093087,0.004557,NaN,-0.008188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C83,-0.054186,NaN,0.029897,0.00028

In [ ]:
for col in TRAIN_DF_RAW.columns:
  print("========================================")
  print(TRAIN_DF_RAW.corr()[col][TRAIN_DF_RAW.corr()[col] > 0.7])
  print("========================================")

In [ ]:
T_corr.to_csv("Train_corr.csv")

In [ ]:
TEST_DF_RAW = dataframe_from_csvs(TEST_DATASET)
TEST_DF_RAW

,timestamp,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,...,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86
0,2021-07-13 16:00:01,-2.1424,0,12.37640,-0.00275,8.02002,0.1524,12.58874,70,1,50,11876,11.68213,28.03598,26979,28.46628,27.40174,0,1,50,27.66017,29.02832,1,298.73773,425.30319,3609,50,16454,1.06824,1,54718,980.07202,51.53651,5.6482,0,108.60724,2880,25.66094,0,0,...,97.25341,1,0,12848,48.20861,1,0.75955,0.32135,0,335.30457,425.87881,6.93170,49.85962,719.92224,1,905,0,0,144,0.18680,9.57111,9972,1,268.33344,35.0,8.72313,236.93082,229.60286,27.68860,48.60640,1.13464,29.09910,2.85153,227.61664,333.51416,1,981.36951,12.0,50,2100
1,2021-07-13 16:00:02,-2.0449,0,12.37640,-0.00087,8.02002,0.1281,12.58874,70,1,50,12094,11.68213,28.03169,26981,28.46628,27.45389,0,1,50,27.66017,29.04358,1,303.80133,425.40543,3656,50,16713,1.08312,1,54719,979.20227,52.49024,5.4256,0,108.60724,2880,25.66094,0,0,...,97.25341,1,0,12896,48.17810,1,0.68719,0.32211,0,340.47668,425.87881,6.93170,49.85962,720.02686,1,905,0,0,144,0.18680,9.60827,9972,1,273.29919,35.0,8.77705,237.31230,229.60286,27.68860,48.62812,1.13312,29.14869,2.85153,231.58908,340.18738,1,981.36951,12.0,50,2165
2,2021-07-13 16:00:03,-2.1287,0,12.37640,-0.00260,8.02002,0.0148,12.58874,70,1,50,12137,11.68213,28.03251,26983,28.46628,27.49862,0,1,50,27.66017,29.04358,1,309.17242,425.40543,3676,50,17022,1.09502,1,54718,980.07202,51.53651,5.1649,0,108.60724,2880,25.66094,0,0,...,97.24579,1,0,12944,48.16284,1,0.63293,0.31906,0,348.79559,425.87881,6.93170,49.85962,720.06177,1,905,0,0,128,0.18680,9.66742,9972,1,268.33344,35.0,8.93085,237.88448,222.65125,27.68860,48.66984,1.13388,29.18836,2.85153,229.60286,347.14990,1,981.36951,12.0,50,2209
3,2021-07-13 16:00:04,-2.1287,0,12.37640,-0.00282,8.02002,0.0148,12.58874,70,1,50,12187,11.68213,28.03251,26985,28.46628,27.56569,0,1,50,27.66017,29.05884,1,313.42230,425.50763,3719,50,17233,1.11219,1,54718,981.37659,51.72729,5.1649,0,108.60724,2880,25.66094,0,0,...,97.25341,1,0,12984,48.13232,1,0.61487,0.32593,0,355.12512,425.87881,6.93170,49.85962,720.06177,1,905,0,0,112,0.18680,9.70509,9972,1,269.32672,35.0,8.91705,237.88448,227.61664,27.68097,48.64198,1.13541,29.24558,2.85153,229.60286,355.90277,1,981.36951,12.0,50,2233
4,2021-07-13 16:00:05,-2.5969,0,12.37640,0.00058,8.02002,-0.2225,12.58874,70,1,50,12198,11.68213,28.03081,26979,28.46628,27.56569,0,1,50,27.66017,29.05884,1,318.25092,424.94534,3725,50,17493,1.11219,1,54718,979.63727,52.10876,4.8932,0,108.60724,2880,25.66094,0,0,...,97.25341,1,0,12696,48.11707,1,0.56064,0.32364,0,358.27185,425.87881,6.93170,49.85962,720.09613,1,906,0,0,96,0.18680,9.77965,9972,1,271.31296,35.0,8.91705,237.12160,230.59618,27.68097,48.59238,1.13541,29.24558,2.85153,222.65125,358.23566,1,981.36951,12.0,50,2290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129595,2021-07-16 23:59:56,-1.8328,0,12.07123,0.00036,15.03143,0.8050,12.58870,70,1,50,16228,18.91479,28.03953,26971,27.86075,28.35412,0,1,50,27.76649,28.95202,1,327.31122,460.57385,386,50,17966,1.04908,1,54718,1010.42865,61.45473,7.1499,0,108.24304,2880,25.68717,0,0,...,28.19976,1,0,4944,35.02502,1,36.85620,1.07132,0,318.08813,467.03854,26.76094,49.89013,719.98303,1,904,0,0,8384,0.06944,29.89865,9917,1,319.97427,35.0,14.31306,250.85446,981.37659,27.83356,49.44022,0.92941,29.45263,2.40398,1056.03259,373.20966,1,1015.06964,12.0,50,22
129596,2021-07-16 23:59:57,-1.8328,0,12.07123,-0.00470,13.74206,0.8050,12.58870,70,1,50,16108,19.18945,28.03953,26970,27.86075,28.47250,0,1,50,27.76649,2

In [ ]:
valid = dataframe_from_csvs(VALIDATION_DATASET)
del valid['timestamp']


,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,...,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,attack
0,-2.0395,0,11.78894,0.00058,16.32080,0.4010,12.903438,70,1,50,10289,11.68213,28.03510,27181,27.84305,29.50472,0,1,50,27.796206,31.04248,1,334.58112,437.98016,3,50,18131,1.04366,1,54830,1032.43347,57.25861,4.9870,0,129.60573,2880,27.736847,0,0,84.15223,...,1,0,0,45.33997,1,71.77734,0.44952,0,292.86029,431.328095,20.989594,51.26343,720.02826,1,927,0,0,15184,0.059445,14.91310,10000,1,300.11276,35.00000,15.09755,264.01520,438.15259,30.90057,50.63332,1.22543,31.52100,0.716042,386.51172,356.64423,1,1034.712769,12.0,50,161,0
1,-2.1750,0,11.78894,-0.00282,15.95459,0.4187,12.903438,70,1,50,10266,11.68213,28.03419,27174,27.84305,29.41619,0,1,50,27.796206,31.04248,1,332.97162,437.62238,2,50,18111,1.04217,1,54829,1032.43347,57.06784,5.3008,0,129.60573,2880,27.736847,0,0,89.49278,...,1,0,0,45.33997,1,74.29114,0.44723,0,295.82605,431.328095,20.989594,51.26343,719.85382,1,926,0,0,14224,0.059445,14.95540,9999,1,298.12650,35.00000,14.71477,264.01520,465.95908,30.88531,50.64244,1.22543,31.51756,0.716042,411.33905,352.08698,1,1034.712769,12.0,50,155,0
2,-2.5854,0,11.78131,-0.00398,14.79492,0.0691,12.903438,70,1,50,10238,11.68213,28.03429,27172,27.84305,29.31749,0,1,50,27.796206,31.04248,1,332.15784,437.11121,2,50,18100,1.03920,1,54828,1031.79565,56.68636,4.5175,0,129.60573,2880,27.736847,0,0,86.82250,...,1,0,0,45.33997,1,70.54761,0.43884,0,300.02173,431.328095,20.989594,51.26343,719.92407,1,927,0,0,12368,0.059445,14.99200,9999,1,297.13321,35.00000,14.30731,263.63373,452.05582,30.58014,50.65156,1.21475,31.50269,0.716042,438.15259,347.63818,1,1034.712769,12.0,50,149,0
3,-2.4722,0,11.78894,-0.00152,14.72626,0.3306,12.903438,70,1,50,10215,11.68213,28.03686,27168,27.84305,29.26632,0,1,50,27.796206,31.04248,1,330.96429,437.16232,2,50,18000,1.03222,1,54829,1033.07117,56.68636,5.4640,0,129.60573,2880,27.736847,0,0,83.38927,...,1,0,8,45.33997,1,59.44373,0.42435,0,302.10144,431.328095,20.989594,51.26343,720.02954,1,924,0,0,11888,0.059445,15.01359,9999,1,295.14697,35.00000,14.54943,264.39661,434.18011,30.06897,50.68747,1.20255,31.47942,0.716042,465.95908,340.33203,1,1034.712769,12.0,50,148,0
4,-2.2319,0,11.78131,0.00072,14.78729,0.2150,12.903438,70,1,50,10085,11.68213,28.02990,27172,27.84305,29.20597,0,1,50,27.796206,31.04248,1,329.02917,437.11121,2,50,17852,1.02947,1,54830,1031.47668,55.73270,4.5850,0,129.60573,2880,27.736847,0,0,79.00238,...,1,0,688,45.33997,1,57.27356,0.40451,0,305.03113,431.328095,20.989594,51.14899,720.02905,1,927,0,0,10688,0.059445,15.04767,9999,1,295.14697,35.00000,15.24461,263.44296,411.33905,30.05371,50.64556,1.20483,31.47942,0.716042,452.05582,335.17798,1,1034.712769,12.0,50,148,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86395,-2.8150,0,12.26196,0.00130,15.53497,-0.0507,12.669310,70,1,50,6494,14.42108,28.02736,27265,27.86172,37.79226,0,1,50,27.796206,31.98853,1,354.96240,455.76889,3,50,19244,1.24152,1,54727,1021.90936,70.03782,5.2654,0,109.91794,2880,27.804180,0,0,335.92218,...,1,0,8,39.84681,1,0.56064,1.38260,0,347.45734,451.941960,20.989594,50.21057,720.00098,1,905,0,0,16,0.193330,35.65305,10022,1,360.69107,25.53188,15.29280,257.72092,1179.76929,38.01117,49.54745,1.25824,31.79814,0.716042,1193.48230,347.58386,1,1018.394040,12.0,50,7,0
86396,-2.6272,0,12.26196,0.00109,15.53497,0.1829,12.669310,70,1,50,6496,14.41345,28.03598,27262,27.86172,37.77182,0,1,50,27.796206,31.98853,1,354.51025,456.22891,3,50,19227,1.24129,1,54725,102

In [ ]:
a_val = valid[valid['attack'] == 1]
a_normal = valid[valid['attack'] == 0]

In [ ]:
for col in valid.columns:
  print("========================================")
  print(a_val.corr()[col][a_val.corr()[col] > 0.7])
  print("========================================")

C01    1.000000
C06    0.786482
C44    0.843975
Name: C01, dtype: float64
Series([], Name: C02, dtype: float64)
C03    1.000000
C84    0.809745
Name: C03, dtype: float64
C04    1.0
Name: C04, dtype: float64
C05    1.000000
C40    0.996488
C67    0.983719
C72    0.997166
C74    0.983151
C80    0.980100
Name: C05, dtype: float64
C01    0.786482
C06    1.000000
C44    0.722337
Name: C06, dtype: float64
C07    1.000000
C11    0.711193
Name: C07, dtype: float64
Series([], Name: C08, dtype: float64)
Series([], Name: C09, dtype: float64)
Series([], Name: C10, dtype: float64)
C07    0.711193
C11    1.000000
Name: C11, dtype: float64
C12    1.000000
C32    0.911358
C43    0.875225
C70    0.918640
Name: C12, dtype: float64
C13    1.0
Name: C13, dtype: float64
C14    1.000000
C21    0.995615
C51    0.745756
Name: C14, dtype: float64
C15    1.000000
C71    0.807585
Name: C15, dtype: float64
C16    1.000000
C75    0.989696
Name: C16, dtype: float64
Series([], Name: C17, dtype: float64)
Series([], N

In [ ]:
for col in valid.columns:
  print("========================================")
  print(a_normal.corr()[col][a_normal.corr()[col] > 0.7])
  print("========================================")

C01    1.000000
C06    0.739419
C44    0.813192
Name: C01, dtype: float64
Series([], Name: C02, dtype: float64)
C03    1.000000
C37    0.907899
Name: C03, dtype: float64
C04    1.0
Name: C04, dtype: float64
C05    1.000000
C40    0.954867
C67    0.940304
C72    0.985222
C74    0.938728
C80    0.928647
Name: C05, dtype: float64
C01    0.739419
C06    1.000000
Name: C06, dtype: float64
C07    1.0
Name: C07, dtype: float64
Series([], Name: C08, dtype: float64)
Series([], Name: C09, dtype: float64)
Series([], Name: C10, dtype: float64)
C11    1.0
Name: C11, dtype: float64
C12    1.000000
C32    0.769874
C43    0.981902
C70    0.767697
Name: C12, dtype: float64
C13    1.0
Name: C13, dtype: float64
C14    1.000000
C21    0.997868
Name: C14, dtype: float64
C15    1.0
Name: C15, dtype: float64
C16    1.000000
C28    0.740140
C75    0.986605
Name: C16, dtype: float64
C17    1.0
C34    1.0
C64    1.0
Name: C17, dtype: float64
Series([], Name: C18, dtype: float64)
Series([], Name: C19, dtype: flo